<a href="https://colab.research.google.com/github/rahiakela/algorithms-for-optimization/blob/main/2-derivatives-and-gradients/2_derivatives_and_gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Derivatives and Gradients


**Optimization is concerned with finding the design point that minimizes (or
maximizes) an objective function.** Knowing how the value of a function changes
as its input is varied is useful because it tells us in which direction we can move to improve on previous points. **The change in the value of the function is measured by the derivative in one dimension and the gradient in multiple dimensions.**

## Setup

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.4.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

**Checking the Installation**

The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.4.2
Commit 44fa15b150* (2020-05-23 18:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, znver1)
Environment:
  JULIA_NUM_THREADS = 2


Now, let's install SymEngine.

In [ ]:
using Pkg
Pkg.add("SymEngine")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed MPC_jll ────────────────────── v1.1.0+0
  Installed OpenSpecFun_jll ────────────── v0.5.3+4
  Installed SymEngine_jll ──────────────── v0.6.0+1
  Installed GMP_jll ────────────────────── v6.1.2+6
  Installed CompilerSupportLibraries_jll ─ v0.3.4+0
  Installed SymEngine ──────────────────── v0.8.3
  Installed SpecialFunctions ───────────── v1.2.1
  Installed ChainRulesCore ─────────────── v0.9.27
  Installed MPFR_jll ───────────────────── v4.0.2+4
######################################################################### 100.0%
######################################################################### 100.0%
######################################################################### 100.0%
######################################################################### 100.0%
######################################################################### 100.0%
######################################################################### 100.0%
   Updating `~/.jul

## Derivatives

The derivative $f^{'}(x)$ of a function $f$ of a single variable x is the rate at which the value of f changes at $x$. It is often visualized, using the tangent line to the graph of the function at $x$. The value of the derivative equals
the slope of the tangent line.

<img src='https://github.com/rahiakela/img-repo/blob/master/algorithms-for-optimization/tangent-line.png?raw=1' width='800'/>

We can use the derivative to provide a linear approximation of the function
near x:

$$f(x + \delta{x}) \approx f(x) + f^{'}(x) \delta{x}$$

The derivative is the ratio between the change in $f$ and the change in $x$ at the point $x$:

$$  f^{'}(x) = \frac{\delta{f(x)}}  {\delta{x}} $$

which is the change in $f(x)$ divided by the change in $x$ as the step becomes
infinitesimally small.

<img src='https://github.com/rahiakela/img-repo/blob/master/algorithms-for-optimization/step-differences.png?raw=1' width='800'/>

The notation $f^{'}(x)$ can be attributed to Lagrange. We also use the notation created by Leibniz,

$$  f^{'}(x) = \frac{df(x)}  {dx} $$

which emphasizes the fact that the derivative is the ratio of the change in $f$ to the change in $x$ at the point $x$.

The limit equation defining the derivative can be presented in three different
ways: 
- the forward difference, 
- the central difference, 
- and the backward difference. 

Each method uses an infinitely small step size $h$:

<img src='https://github.com/rahiakela/img-repo/blob/master/algorithms-for-optimization/symbolic-differentiation.png?raw=1' width='800'/>

If $f$ can be represented symbolically, symbolic differentiation can often provide an exact analytic expression for $f^{'}$ by applying derivative rules from calculus. The analytic expression can then be evaluated at any point $x$.




In [ ]:
using SymEngine

# define x as a symbolic variable
@vars x;
f = x ^ 2 + x / 2 - sin(x) / x;
diff(f, x)

1/2 + 2*x + sin(x)/x^2 - cos(x)/x

## Derivatives in Multiple Dimensions